<a href="https://colab.research.google.com/github/mohd-faizy/CAREER-TRACK-Machine-Learning-Scientist-with-Python/blob/main/04_Dealing_with_Text_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

--- 
<strong> 
    <h1 align='center'>Dealing with Text Data</h1> 
</strong>

---
> Finally, in this chapter, you will work with **unstructured text data**, understanding ways in which you can **engineer column features** out of a text corpus. You will compare how different approaches may impact how much context is being extracted from a text, and how to balance the need for context, without too many features being created.


In [25]:
!git clone https://github.com/mohd-faizy/CAREER-TRACK-Machine-Learning-Scientist-with-Python.git

Cloning into 'CAREER-TRACK-Machine-Learning-Scientist-with-Python'...
remote: Enumerating objects: 275, done.
remote: Counting objects: 100% (275/275), done.
remote: Compressing objects: 100% (248/248), done.
remote: Total 604 (delta 53), reused 240 (delta 25), pack-reused 329
Receiving objects: 100% (604/604), 198.75 MiB | 35.08 MiB/s, done.
Resolving deltas: 100% (179/179), done.
Checking out files: 100% (296/296), done.


In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

plt.style.use('fivethirtyeight')
#plt.style.use('ggplot')
#sns.set_theme()

%matplotlib inline

In [27]:
os.chdir('/content/CAREER-TRACK-Machine-Learning-Scientist-with-Python/09_Feature_Engineering_for_Machine_Learning_in_Python/_dataset')
cwd = os.getcwd()
print('Curent working directory is ', cwd)

Curent working directory is  /content/CAREER-TRACK-Machine-Learning-Scientist-with-Python/09_Feature_Engineering_for_Machine_Learning_in_Python/_dataset


In [28]:
ls

CAREER-TRACK-Machine-Learning-Scientist-with-Python/  inaugural_speeches.csv
Combined_DS_v10.csv


## **Encoding text**


### Cleaning up your text
Unstructured text data cannot be directly used in most analyses. Multiple steps need to be taken to go from a long free form string to a set of numeric columns in the right format that can be ingested by a machine learning model. The first step of this process is to standardize the data and eliminate any characters that could cause problems later on in your analytic pipeline.

In this chapter you will be working with a new dataset containing the inaugural speeches of the presidents of the United States loaded as `speech_df`, with the speeches stored in the `text` column.

In [29]:
speech_df = pd.read_csv('inaugural_speeches.csv')
speech_df.head()

,Name,Inaugural Address,Date,text
0,George Washington,First Inaugural Address,"Thursday, April 30, 1789",Fellow-Citizens of the Senate and of the House...
1,George Washington,Second Inaugural Address,"Monday, March 4, 1793",Fellow Citizens: I AM again called upon by th...
2,John Adams,Inaugural Address,"Saturday, March 4, 1797","WHEN it was first perceived, in early times, t..."
3,Thomas Jefferson,First Inaugural Address,"Wednesday, March 4, 1801",Friends and Fellow-Citizens: CALLED upon to u...
4,Thomas Jefferson,Second Inaugural Address,"Monday, March 4, 1805","PROCEEDING, fellow-citizens, to that qualifica..."


In [30]:
# Print the first 5 rows of the text column
speech_df['text'].head()

0    Fellow-Citizens of the Senate and of the House...
1    Fellow Citizens:  I AM again called upon by th...
2    WHEN it was first perceived, in early times, t...
3    Friends and Fellow-Citizens:  CALLED upon to u...
4    PROCEEDING, fellow-citizens, to that qualifica...
Name: text, dtype: object

In [31]:
# Replace all non-letter char with a whitespace with patterns called regular expressions(regex)
speech_df['text_clean'] = speech_df['text'].str.replace('[^a-zA-Z]', ' ')

# Change to lower case
speech_df['text_clean'] = speech_df['text_clean'].str.lower()

# Print the first 5 rows of text_clean column
print(speech_df['text_clean'].head())

0    fellow citizens of the senate and of the house...
1    fellow citizens   i am again called upon by th...
2    when it was first perceived  in early times  t...
3    friends and fellow citizens   called upon to u...
4    proceeding  fellow citizens  to that qualifica...
Name: text_clean, dtype: object


### High level text features
Once the text has been cleaned and standardized you can begin creating features from the data. The most fundamental information you can calculate about free form text is its size, such as its length and number of words. In this exercise (and the rest of this chapter), you will focus on the cleaned/transformed text column (`text_clean`) you created in the last exercise.



In [32]:
# Find the length of each text
speech_df['char_cnt'] = speech_df['text_clean'].str.len()

# Count the number of words in each text
speech_df['word_cnt'] = speech_df['text_clean'].str.split().str.len()

# Find the average length of word
speech_df['avg_word_length'] = speech_df['char_cnt'] / speech_df['word_cnt']

# Print the first 5 rows of these columns
speech_df[['text_clean', 'char_cnt', 'word_cnt', 'avg_word_length']].head()

,text_clean,char_cnt,word_cnt,avg_word_length
0,fellow citizens of the senate and of the house...,8616,1432,6.016760
1,fellow citizens i am again called upon by th...,787,135,5.829630
2,when it was first perceived in early times t...,13871,2323,5.971158
3,friends and fellow citizens called upon to u...,10144,1736,5.843318
4,proceeding fellow citizens to that qualifica...,12902,2169,5.948363


## **Word counts**


### Counting words (I)

Once high level information has been recorded you can begin creating features based on the actual content of each text. One way to do this is to approach it in a similar way to how you worked with categorical variables in the earlier lessons.

- For each unique word in the dataset a column is created.
- For each entry, the number of times this word occurs is counted and the count value is entered into the respective column.

These `"count"` columns can then be used to train machine learning models.

In [33]:
from sklearn.feature_extraction.text import CountVectorizer

# Instantiate CountVectorizer
cv = CountVectorizer()

# Fit the vectorizer
cv.fit(speech_df['text_clean'])

# Print feature names
print(cv.get_feature_names()[:10])

['abandon', 'abandoned', 'abandonment', 'abate', 'abdicated', 'abeyance', 'abhorring', 'abide', 'abiding', 'abilities']


>This **vectorizer** can be applied to both the text it was trained on, and new texts.

### Counting words (II)
Once the vectorizer has been fit to the data, it can be used to transform the text to an array representing the word counts. This array will have a row per block of text and a column for each of the features generated by the vectorizer that you observed in the last exercise.

In [34]:
# Apply the vectorizer
cv_transformed = cv.transform(speech_df['text_clean'])

# Print the full array
cv_array = cv_transformed.toarray()
print(cv_array)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [35]:
# Print the shape of cv_array
print(cv_array.shape)

(58, 9043)


The speeches have 9043 unique words, which is a lot! we have to limit these set of features.

### Limiting your features
As you have seen, using the __`CountVectorizer`__ -:  $\color{blue}{\text{with default settings creates a feature for every single word in your corpus.}}$ This can create far too many features, often including ones that will provide very little analytical value.

For this purpose __`CountVectorizer`__ has parameters that you can set to reduce the number of features:-

- __`min_df`__ : Use only words that occur in more than this percentage of documents. This can be used to remove outlier words that will not generalize across texts.
- __`max_df`__ : Use only words that occur in less than this percentage of documents. This is useful to eliminate very common words that occur in every corpus without adding value such as "and" or "the".

In [36]:
# Import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Specify arguements to limit the number of features generated
cv = CountVectorizer(min_df=0.2, max_df=0.8)

# Fit, transform, and convert into array
cv_transformed = cv.fit_transform(speech_df['text_clean'])
cv_array = cv_transformed.toarray()

# Print the array shape
print(cv_array.shape)

(58, 818)


The number of features(*unique words*) greatly reduced from $9043$ to $818$?

In [37]:
cv_array

array([[ 0,  0,  0, ...,  5,  0,  9],
       [ 0,  0,  0, ...,  0,  0,  1],
       [ 0,  0,  0, ...,  0,  0,  1],
       ...,
       [ 0,  1,  0, ..., 14,  1,  3],
       [ 0,  0,  0, ...,  5,  1,  0],
       [ 0,  0,  0, ..., 14,  1, 11]])

### Text to DataFrame
Now that you have generated these count based features in an array you will need to reformat them so that they can be combined with the rest of the dataset. This can be achieved by converting the array into a pandas DataFrame, with the feature names you found earlier as the column names, and then concatenate it with the original DataFrame.

In [38]:
# Create a DataFrame with these features
cv_df = pd.DataFrame(cv_array, columns = cv.get_feature_names()).add_prefix('Counts_')

# Add the new columns to the original DataFrame
speech_df_new = pd.concat([speech_df, cv_df], axis=1, sort=False)
speech_df_new.head()

,Name,Inaugural Address,Date,text,text_clean,char_cnt,word_cnt,avg_word_length,Counts_abiding,Counts_ability,Counts_able,Counts_about,Counts_above,Counts_abroad,Counts_accept,Counts_accomplished,Counts_achieve,Counts_across,Counts_act,Counts_action,Counts_acts,Counts_add,Counts_adequate,Counts_administration,Counts_adopted,Counts_advance,Counts_advantage,Counts_affairs,Counts_afford,Counts_after,Counts_again,Counts_against,Counts_age,Counts_ago,Counts_agriculture,Counts_aid,Counts_alike,Counts_almighty,Counts_almost,Counts_alone,...,Counts_vital,Counts_voice,Counts_want,Counts_war,Counts_wars,Counts_washington,Counts_waste,Counts_way,Counts_ways,Counts_weak,Counts_wealth,Counts_weight,Counts_welfare,Counts_were,Counts_what,Counts_whatever,Counts_where,Counts_wherever,Counts_whether,Counts_while,Counts_whole,Counts_whom,Counts_whose,Counts_willing,Counts_wisdom,Counts_wise,Counts_wisely,Counts_wish,Counts_within,Counts_without,Counts_women,Counts_words,Counts_work,Counts_wrong,Counts_year,Counts_years,Counts_yet,Counts_you,Counts_young,Counts_your
0,George Washington,First Inaugural Address,"Thursday, April 30, 1789",Fellow-Citizens of the Senate and of the House...,fellow citizens of the senate and of the house...,8616,1432,6.016760,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,2,0,0,1,1,0,0,1,1,0,0,0,0,0,1,0,0,...,0,2,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0,0,0,2,0,0,0,0,0,1,0,5,0,9
1,George Washington,Second Inaugural Address,"Monday, March 4, 1793",Fellow Citizens: I AM again called upon by th...,fellow citizens i am again called upon by th...,787,135,5.829630,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,John Adams,Inaugural Address,"Saturday, March 4, 1797","WHEN it was first perceived, in early times, t...",when it was first perceived in early times t...,13871,2323,5.971158,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,3,1,0,1,0,0,3,0,1,1,0,1,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0,1,0,1,5,3,2,1,0,0,1,3,0,0,0,0,1,0,1,0,3,0,0,0,0,2,3,0,0,0,1
3,Thomas Jefferson,First Inaugural Address,"Wednesday, March 4, 1801",Friends and Fellow-Citizens: CALLED upon to u...,friends and fellow citizens called upon to u...,10144,1736,5.843318,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,2,0,2,0,1,0,0,0,2,1,0,1,0,0,0,0,1,...,2,1,1,1,0,0,0,0,0,0,0,0,0,0,5,1,3,0,0,0,2,1,2,0,2,1,0,1,1,2,0,0,1,2,0,0,2,7,0,7
4,Thomas Jefferson,Second Inaugural Address,"Monday, March 4, 1805","PROCEEDING, fellow-citizens, to that qualifica...",proceeding fellow citizens to that qualifica...,12902,2169,5.948363,0,0,1,0,0,0,0,0,0,0,3,1,0,1,0,1,0,1,0,3,1,0,2,6,0,0,1,0,0,0,0,2,...,0,0,1,3,1,0,0,0,0,0,0,1,0,0,4,0,1,0,3,0,1,2,3,0,2,0,0,1,4,2,0,0,0,0,2,2,2,4,0,4


## Term frequency-inverse document frequency(TF-IDF)

TF-IDF are word frequency scores that try to highlight words that are more interesting, e.g. frequent in a document but not across documents. The TfidfVectorizer will tokenize documents, learn the vocabulary and inverse document frequency weightings, and allow you to encode new documents.

$$\color{blue}{\textbf{TF-IDF}} = \frac{\frac{\text{count of word occurances}}{\textbf{Total words in documents}}}{\log (\frac{\text{Number of docs word is in}}{\textbf{Total number of docs}})} $$

TF-IDF divides ***number of times a word occurs in the document*** by a measure of ***what proportion of the documents a word occurs in all documents***. This has the effect of reducing the value of common words, while increasing the weight of words that do not occur in many documents.

**How is TfidfVectorizer different than CountVectorizer?**

- __`TfidfVectorizer()`__ **returns** __`float`__ while the __`CountVectorizer()`__ returns __`int`__.

- __`TfidfVectorizer()`__ assigns a **score** while __`CountVectorizer()`__ **counts**.

### TF-IDF

While counts of occurrences of words can be useful to build models, words that occur many times may **skew** the results undesirably.

To **limit these common words from overpowering** your model a form of normalization can be used. we use Term frequency-inverse document frequency (Tf-idf).

> - ${\textbf{Tf-idf has the effect of reducing the value of common words}}$
- ${\textbf{While increasing the weight of words that do not occur in many documents.}}$



>we can specify the **maximum number of features** using `max_features` which will only use the $100$ most common words. We will also specify the vectorizer to omit a set of `stop_words`, these are a **predefined list of the most common english** words such as ___"and" or "the"___. You can use scikit-learn's built in list, load your own, or use lists provided by other python libraries.

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate TfidfVectorizer
tfidf = TfidfVectorizer(max_features=100, stop_words='english')

# Fit the vectorizer and transform the data
tfidf_transformed = tfidf.fit_transform(speech_df['text_clean'])

# Create a DataFrame with these features
tfidf_df = pd.DataFrame(tfidf_transformed.toarray(),
                     columns=tfidf.get_feature_names()).add_prefix('TFIDF_')
tfidf_df.head()

,TFIDF_action,TFIDF_administration,TFIDF_america,TFIDF_american,TFIDF_americans,TFIDF_believe,TFIDF_best,TFIDF_better,TFIDF_change,TFIDF_citizens,TFIDF_come,TFIDF_common,TFIDF_confidence,TFIDF_congress,TFIDF_constitution,TFIDF_country,TFIDF_day,TFIDF_duties,TFIDF_duty,TFIDF_equal,TFIDF_executive,TFIDF_faith,TFIDF_far,TFIDF_federal,TFIDF_fellow,TFIDF_force,TFIDF_foreign,TFIDF_free,TFIDF_freedom,TFIDF_future,TFIDF_general,TFIDF_god,TFIDF_good,TFIDF_government,TFIDF_great,TFIDF_high,TFIDF_history,TFIDF_home,TFIDF_hope,TFIDF_human,...,TFIDF_need,TFIDF_new,TFIDF_office,TFIDF_old,TFIDF_order,TFIDF_party,TFIDF_peace,TFIDF_people,TFIDF_place,TFIDF_policy,TFIDF_political,TFIDF_power,TFIDF_powers,TFIDF_present,TFIDF_president,TFIDF_principles,TFIDF_progress,TFIDF_prosperity,TFIDF_public,TFIDF_purpose,TFIDF_right,TFIDF_rights,TFIDF_secure,TFIDF_service,TFIDF_shall,TFIDF_spirit,TFIDF_state,TFIDF_states,TFIDF_strength,TFIDF_support,TFIDF_things,TFIDF_time,TFIDF_today,TFIDF_union,TFIDF_united,TFIDF_war,TFIDF_way,TFIDF_work,TFIDF_world,TFIDF_years
0,0.000000,0.133415,0.000000,0.105388,0.0,0.000000,0.000000,0.000000,0.000000,0.229644,0.0,0.000000,0.111079,0.000000,0.060755,0.229644,0.115098,0.064225,0.238637,0.063036,0.147280,0.000000,0.178978,0.000000,0.147528,0.000000,0.000000,0.098352,0.000000,0.101797,0.000000,0.0,0.147528,0.367430,0.133183,0.000000,0.000000,0.000000,0.051787,0.126073,...,0.00000,0.049176,0.000000,0.000000,0.141458,0.070729,0.000000,0.174590,0.056532,0.138691,0.000000,0.050898,0.065448,0.315182,0.061880,0.063036,0.000000,0.064225,0.333237,0.0,0.055540,0.050898,0.000000,0.063036,0.145021,0.000000,0.000000,0.103573,0.000000,0.000000,0.000000,0.045929,0.0,0.136012,0.203593,0.000000,0.060755,0.000000,0.045929,0.052694
1,0.000000,0.261016,0.266097,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.179712,0.0,0.000000,0.217318,0.000000,0.237725,0.179712,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.192418,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.179712,0.000000,0.233437,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.242128,0.000000,0.000000,0.000000,0.000000,0.170786,0.000000,0.000000,0.000000,0.000000,0.000000,0.246652,0.242128,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.567446,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.199157,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.092436,0.157058,0.073018,0.0,0.000000,0.026112,0.060460,0.000000,0.106072,0.0,0.056125,0.025654,0.196017,0.224501,0.212143,0.026582,0.029665,0.055113,0.058233,0.068028,0.082669,0.027556,0.000000,0.068143,0.000000,0.246496,0.045428,0.000000,0.023510,0.133321,0.0,0.136285,0.339429,0.102528,0.027556,0.029116,0.000000,0.023920,0.058233,...,0.00000,0.022714,0.000000,0.000000,0.130678,0.130678,0.121696,0.403213,0.026112,0.000000,0.027556,0.117549,0.030230,0.058233,0.000000,0.058233,0.000000,0.059331,0.153921,0.0,0.025654,0.023510,0.030230,0.058233,0.089313,0.153921,0.090691,0.215280,0.000000,0.116465,0.032030,0.021214,0.0,0.062823,0.070529,0.024339,0.000000,0.000000,0.063643,0.073018
3,0.000000,0.092693,0.000000,0.000000,0.0,0.090942,0.117831,0.045471,0.053335,0.223369,0.0,0.084421,0.154348,0.000000,0.084421,0.127639,0.039983,0.089243,0.000000,0.175183,0.051163,0.082899,0.041449,0.059596,0.239161,0.048179,0.000000,0.102498,0.171970,0.035363,0.100268,0.0,0.170829,0.382918,0.030844,0.124348,0.087591,0.045471,0.035980,0.000000,...,0.00000,0.000000,0.042992,0.000000,0.049140,0.000000,0.183051,0.060650,0.039277,0.000000,0.124348,0.141450,0.045471,0.000000,0.000000,0.131387,0.000000,0.044621,0.154348,0.0,0.154348,0.070725,0.000000,0.000000,0.201512,0.000000,0.090942,0.000000,0.000000,0.131387,0.048179,0.000000,0.0,0.094497,0.000000,0.036610,0.000000,0.039277,0.095729,0.000000
4,0.041334,0.039761,0.000000,0.031408,0.0,0.000000,0.067393,0.039011,0.091514,0.273760,0.0,0.000000,0.033105,0.000000,0.217280,0.109504,0.034302,0.1531

Did you notice that counting the word occurences and calculating the Tf-idf weights are very similar? This is one of the reasons scikit-learn is very popular, a consistent API.

### Inspecting Tf-idf values
After creating Tf-idf features you will often want to understand what are the most highest scored words for each corpus. This can be achieved by isolating the row you want to examine and then sorting the the scores from high to low.



In [40]:
# Isolate the row to be examined
sample_row = tfidf_df.iloc[0]

# Print the top 5 words of the sorted output
print(sample_row.sort_values(ascending=False).head())

TFIDF_government    0.367430
TFIDF_public        0.333237
TFIDF_present       0.315182
TFIDF_duty          0.238637
TFIDF_citizens      0.229644
Name: 0, dtype: float64


### Transforming unseen data

When creating vectors from text, any transformations that you perform before training a machine learning model, you also need to apply on the new unseen (test) data. To achieve this follow the same approach from the last chapter: fit the vectorizer only on the training data, and apply it to the test data.

For this exercise the `speech_df` DataFrame has been split in two:

- `train_speech_df`: The training set consisting of the first 45 speeches.
- `test_speech_df`: The test set consisting of the remaining speeches.

**Applying the vectorizer to new data**

So how do you apply this transformation on the test set? As mentioned before, you should preprocess your test data using the transformations made on the train data only. **To ensure that the same features are created you should use the same vectorizer that you fit on the training data.** 

So first **transform** the test data using the ***tfidf vectorizer*** and then **recreate** the test data set by combining the __`tfidf.get_feature_names())`__ and other columns.

In [41]:
train_speech_df = speech_df.iloc[:45]
test_speech_df = speech_df.iloc[45:]

In [42]:
# Instantiate TfidfVectorizer
tfidf = TfidfVectorizer(max_features=100, stop_words='english')

# Fit the vectorizer and transform the data
tfidf_transformed = tfidf.fit_transform(train_speech_df['text_clean'])

# Transform test data
test_tfidf_transformed = tfidf.transform(test_speech_df['text_clean'])

# Create new features for the test set
test_tfidf_df = pd.DataFrame(test_tfidf_transformed.toarray(), 
                          columns=tfidf.get_feature_names()).add_prefix('TFIDF_')
test_tfidf_df.head()

,TFIDF_action,TFIDF_administration,TFIDF_america,TFIDF_american,TFIDF_authority,TFIDF_best,TFIDF_business,TFIDF_citizens,TFIDF_commerce,TFIDF_common,TFIDF_confidence,TFIDF_congress,TFIDF_constitution,TFIDF_constitutional,TFIDF_country,TFIDF_day,TFIDF_duties,TFIDF_duty,TFIDF_equal,TFIDF_executive,TFIDF_faith,TFIDF_far,TFIDF_federal,TFIDF_fellow,TFIDF_force,TFIDF_foreign,TFIDF_free,TFIDF_freedom,TFIDF_future,TFIDF_general,TFIDF_given,TFIDF_god,TFIDF_good,TFIDF_government,TFIDF_great,TFIDF_high,TFIDF_hope,TFIDF_human,TFIDF_important,TFIDF_institutions,...,TFIDF_order,TFIDF_ought,TFIDF_party,TFIDF_peace,TFIDF_people,TFIDF_policy,TFIDF_political,TFIDF_power,TFIDF_powers,TFIDF_present,TFIDF_principle,TFIDF_principles,TFIDF_progress,TFIDF_proper,TFIDF_prosperity,TFIDF_protection,TFIDF_public,TFIDF_purpose,TFIDF_question,TFIDF_republic,TFIDF_revenue,TFIDF_right,TFIDF_rights,TFIDF_secure,TFIDF_self,TFIDF_service,TFIDF_shall,TFIDF_spirit,TFIDF_state,TFIDF_states,TFIDF_subject,TFIDF_support,TFIDF_time,TFIDF_union,TFIDF_united,TFIDF_war,TFIDF_way,TFIDF_work,TFIDF_world,TFIDF_years
0,0.000000,0.029540,0.233954,0.082703,0.000000,0.000000,0.000000,0.022577,0.0,0.000000,0.026350,0.000000,0.026950,0.0,0.022577,0.029540,0.000000,0.0,0.065003,0.0,0.031720,0.056409,0.000000,0.049296,0.000000,0.000000,0.049296,0.066626,0.026350,0.000000,0.030968,0.195008,0.024111,0.115378,0.110450,0.055135,0.079050,0.000000,0.000000,0.0,...,0.034158,0.0,0.000000,0.316200,0.302600,0.0,0.000000,0.025767,0.000000,0.000000,0.000000,0.000000,0.030968,0.0,0.0,0.0,0.000000,0.029540,0.000000,0.000000,0.0,0.000000,0.000000,0.030242,0.000000,0.0,0.086457,0.165406,0.000000,0.024648,0.0,0.000000,0.115378,0.000000,0.024648,0.079050,0.033313,0.000000,0.299983,0.134749
1,0.000000,0.000000,0.547457,0.036862,0.000000,0.036036,0.000000,0.015094,0.0,0.000000,0.017617,0.000000,0.000000,0.0,0.045283,0.019750,0.000000,0.0,0.021730,0.0,0.084830,0.037714,0.000000,0.016479,0.043459,0.000000,0.000000,0.089089,0.052851,0.000000,0.020704,0.086919,0.016120,0.154278,0.132920,0.018431,0.035234,0.040438,0.043459,0.0,...,0.022837,0.0,0.000000,0.334722,0.086705,0.0,0.000000,0.017227,0.018857,0.000000,0.024041,0.000000,0.103522,0.0,0.0,0.0,0.000000,0.019750,0.024685,0.000000,0.0,0.108108,0.016120,0.020219,0.000000,0.0,0.101155,0.036862,0.000000,0.000000,0.0,0.019296,0.092567,0.000000,0.000000,0.052851,0.066817,0.078999,0.277701,0.126126
2,0.000000,0.000000,0.126987,0.134669,0.000000,0.131652,0.000000,0.000000,0.0,0.046997,0.042907,0.000000,0.000000,0.0,0.036763,0.048102,0.045927,0.0,0.052924,0.0,0.103304,0.000000,0.000000,0.000000,0.000000,0.049244,0.040136,0.216981,0.085814,0.000000,0.100853,0.052924,0.078521,0.150301,0.071941,0.089780,0.085814,0.246220,0.000000,0.0,...,0.000000,0.0,0.000000,0.042907,0.211174,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.093993,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.043884,0.117781,0.000000,0.054245,0.0,0.000000,0.269339,0.000000,0.040136,0.0,0.000000,0.075151,0.000000,0.080272,0.042907,0.054245,0.096203,0.225452,0.043884
3,0.037094,0.067428,0.267012,0.031463,0.039990,0.061516,0.050085,0.077301,0.0,0.000000,0.000000,0.039990,0.030758,0.0,0.077301,0.134856,0.000000,0.0,0.074188,0.0,0.108607,0.032190,0.183116,0.084393,0.000000,0.000000,0.056262,0.304162,0.090220,0.000000,0.000000,0.185469,0.027517,0.421380,0.100845,0.031463,0.060146,0.034515,0.037094,0.0,...,0.038984,0.0,0.038984,0.060146,0.222015,0.0,0.092274,0.029408,0.032190,0.094389,0.000000,0.032940,0.070687,0.0,0.0,0.0,0.029408,0.000000,0.000000,0.039990,0.0,0.030758,0.000000,0.000000,0.076040,0.0,0.024668,0.000000,0.000000,0.112524,0.0,0.098819,0.210690,0.000000,0.056262,0.030073,0.038020,0.235998,0.237026,0.061516
4,0.000000,0.000000,0.221561,0.156644,0.028442,0.087505,0.000000,0.109959,0.0,0.023428,0.021389,0.028442,0.000000,0.0,0.018327,0.143872,0.000000,0.0,0.026383,0.0,0.077246,0.000000,0.162799,0.060023,0.000000,0.000000,0.060023,0.378580,0.042778,0.025138,0.025138,0.211061,

## Bag of words and N-grams

- In the **Bag of words** each words is treated as if they are being **drawn from a bag with no concept of order or grammar**. While analyzing the occurrences of individual words can be a valuable way to create features from a piece of text, you will notice that **individual words can loose all their context/meaning when viewed independently**.


**Issues with bag of words**
- **Positive meaning**
    - ***Single word***: happy

- **Negative meaning**
    - ***Bi-gram*** : not happy

- **Positive meaning**
    - ***Trigram***: never not happy

One common method to retain at least some concept of word order in a text is to instead use multiple consecutive words like pairs (**bi-gram**) or three consecutive words (**tri-grams**). **This maintains at least some ordering information while at the same time allowing for the creation of a reasonable set of features.**




**Using N-grams**

To leverage `n-grams` in your own models an additional argument "ngram_range", can be specified when instantiating your **TF-IDF vectorizer**. The values assigned to the argument are the minimum and maximum length of **n-grams** to be included. In this case you would only be looking at bi-grams (**n-grams** with two words) Printing the **bi-gram** features created we can see the pairs of words instead of single words.

### **TfidfVectorizer**

### **Tri-Grams**

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Using N-grams
Tfidf_tri_gram_vec = TfidfVectorizer(ngram_range = (3,3))

# fit_transform and apply bigram vectorizer
Tfidf_tri_gram = Tfidf_tri_gram_vec.fit_transform(speech_df['text'])

# Print the bigram features
print(Tfidf_tri_gram_vec.get_feature_names())

['0085 it 0094', '0085 that in', '0085 that the', '0085 the dangerous', '0085 the idealistic', '0092 author who', '0092 belief in', '0092 birth in', '0092 borders while', '0092 bright flame', '0092 capital has', '0092 capital there', '0092 cause my', '0092 ceremony however', '0092 children that', '0092 constant curse', '0092 courage to', '0092 course you', '0092 decline is', '0092 defeat today', '0092 dollars will', '0092 dream was', '0092 enemies the', '0092 faith in', '0092 going to', '0092 grace upon', '0092 grand story', '0092 greatest democracy', '0092 growing middle', '0092 how we', '0092 ideal of', '0092 ills can', '0092 ills on', '0092 indispensable nation', '0092 influence is', '0092 infrastructure has', '0092 journey the', '0092 leaders have', '0092 ll need', '0092 mightiest industrial', '0092 mission america', '0092 most prized', '0092 most productive', '0092 people live', '0092 permanent under', '0092 possibilities are', '0092 prayer church', '0092 promise and', '0092 promi

#### **Finding common words**

when creating new features, you should always take time to check your work, and ensure that the features you are creating make sense. A good way to check your `n-grams` is to see what are the most common values being recorded. This can be done by summing the values of your DataFrame of count values that you created using the `**sum**()` method.

In [44]:
# Create a DataFrame with the Counts features
Tfidf_df = pd.DataFrame(Tfidf_tri_gram.toarray(),
                     columns=Tfidf_tri_gram_vec.get_feature_names()).add_prefix('Counts_')
Tfidf_sums = Tfidf_df.sum()
print(Tfidf_sums.head())

Counts_0085 it 0094           0.020802
Counts_0085 that in           0.020802
Counts_0085 that the          0.020802
Counts_0085 the dangerous     0.022294
Counts_0085 the idealistic    0.022294
dtype: float64


In [45]:
Tfidf_sums.sort_values(ascending=False).head()

Counts_the united states    0.947748
Counts_of the united        0.659646
Counts_of the people        0.653524
Counts_of the government    0.636495
Counts_of the world         0.529463
dtype: float64

It comes as no surprise that the most commonly occurring tri-gram in a dataset of US president's speeches is "**United States"** which indicates that the features being created make sense.



### **CountVectorizer**

#### Using longer n-grams
So far you have created features based on individual words in each of the texts. This can be quite powerful when used in a machine learning model but you may be concerned that by looking at words individually a lot of the context is being ignored. To deal with this when creating models you can use n-grams which are sequence of n words grouped together. For example:

- bigrams: Sequences of two consecutive words
- trigrams: Sequences of two consecutive words

These can be automatically created in your dataset by specifying the ngram_range argument as a tuple `(n1, n2)` where all n-grams in the `n1` to `n2` range are included.

In [46]:
from sklearn.feature_extraction.text import CountVectorizer

# Instantiate a trigram vectorizer
cv_trigram_vec = CountVectorizer(max_features=100, 
                                 stop_words='english', 
                                 ngram_range=(3, 3))

# Fit and apply trigram vectorizer
cv_trigram = cv_trigram_vec.fit_transform(speech_df['text_clean'])

# Print the trigram features
cv_trigram_vec.get_feature_names()[:10]

['ability preserve protect',
 'agriculture commerce manufactures',
 'america ideal freedom',
 'amity mutual concession',
 'anchor peace home',
 'ask bow heads',
 'best ability preserve',
 'best interests country',
 'bless god bless',
 'bless united states']

#### Finding the most common words
Its always advisable once you have created your features to inspect them to ensure that they are as you would expect. This will allow you to catch errors early, and perhaps influence what further feature engineering you will need to do.

In [47]:
# Create a DataFrame of the features
cv_tri_df = pd.DataFrame(cv_trigram.toarray(), 
                        columns = cv_trigram_vec.get_feature_names()).add_prefix('Counts_')

# Print the top 5 words in the sorted output
cv_tri_df.sum().sort_values(ascending=False).head()

Counts_constitution united states    20
Counts_people united states          13
Counts_preserve protect defend       10
Counts_mr chief justice              10
Counts_president united states        8
dtype: int64

## __Connect with Me__ 
--- 
[<img align="left" alt="codeSTACKr | Twitter" width="40px" src="https://cdn.jsdelivr.net/npm/simple-icons@v3/icons/twitter.svg" />][twitter] 
[<img align="left" alt="codeSTACKr | LinkedIn" width="40px" src="https://cdn.jsdelivr.net/npm/simple-icons@v3/icons/linkedin.svg" />][linkedin] 
[<img align="left" alt="codeSTACKr.com" width="40px" src="https://raw.githubusercontent.com/iconic/open-iconic/master/svg/globe.svg" />][StackExchange AI] 
[twitter]: https://twitter.com/F4izy 
--- 
[linkedin]: https://www.linkedin.com/in/mohd-faizy/ 
--- 
[StackExchange AI]: https://mohd-faizy.github.io
--- 
--- 
--- 
---